<a href="https://colab.research.google.com/github/lburiti/aprendizado-machine-learning-Python/blob/Deep_Learning/Rede_Neural_do_zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>